### Calculo de la huella de crabono usada en todo nuestro codigo

In [ ]:
pip install -q --upgrade numpy pandas codecarbon enum34 mkl jupyterlab tqdm openpyxl

In [2]:
pue = 1.12
country_iso_code = 'ESP'
region='ESP'
cloud_provider='gcp'
cloud_region='europe-southwest1'
country_2letter_iso_code = 'ES'

# Para este caso, únicamnete se definirá el nombre del fichero de salida. Si el usuario lo desea, puede añadir otros.
output_file = 'test_emissions.csv'

In [3]:
from codecarbon import OfflineEmissionsTracker
tracker = OfflineEmissionsTracker(
        country_iso_code=country_iso_code,
        region=region,
        cloud_provider=cloud_provider,
        cloud_region=cloud_region,
        country_2letter_iso_code=country_2letter_iso_code,
        pue=pue,
        output_file = output_file
        )

[codecarbon INFO @ 20:22:28] offline tracker init
[codecarbon WARNING @ 20:22:28] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 20:22:28] [setup] RAM Tracking...
[codecarbon INFO @ 20:22:28] [setup] CPU Tracking...
[codecarbon WARNING @ 20:22:30] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Linux OS detected: Please ensure RAPL files exist at /sys/class/powercap/intel-rapl/subsystem to measure CPU

[codecarbon INFO @ 20:22:30] CPU Model on constant consumption mode: Intel(R) Xeon(R) Platinum 8370C CPU @ 2.80GHz
[codecarbon WARNING @ 20:22:30] No CPU tracking mode found. Falling back on CPU load mode.
[codecarbon INFO @ 20:22:30] [setup] GPU Tracking...
[codecarbon INFO @ 20:22:30] No GPU found.
[codecarbon INFO @ 20:22:30] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: cpu_load
                GPU Tracking Method

In [4]:
# Inicio del proceso de medición
tracker.start()

In [8]:
import os
import pandas as pd
import requests
import zipfile
import shutil
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm

[codecarbon INFO @ 20:23:05] Energy consumed for RAM : 0.000579 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 20:23:06] Delta energy consumed for CPU with cpu_load : 0.000185 kWh, power : 41.0 W
[codecarbon INFO @ 20:23:06] Energy consumed for All CPU : 0.000383 kWh
[codecarbon INFO @ 20:23:06] 0.000961 kWh of electricity used since the beginning.


Inaturalist

In [9]:
url = "https://static.inaturalist.org/observations/gbif-observations-dwca.zip"
local_zip_path = "gbif-observations-dwca.zip"
extract_path = "Datos iNaturalist"

print("Downloading file...")
with requests.get(url, stream=True) as r:
    r.raise_for_status()
    total_size = int(r.headers.get('content-length', 0))
    with open(local_zip_path, "wb") as f, tqdm(
        total=total_size, unit='B', unit_scale=True, unit_divisor=1024, desc=local_zip_path
    ) as pbar:
        for chunk in r.iter_content(chunk_size=8192):
            f.write(chunk)
            pbar.update(len(chunk))

print(f"Extracting to '{extract_path}'...")
os.makedirs(extract_path, exist_ok=True)
with zipfile.ZipFile(local_zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

os.remove(local_zip_path)
print("Deleted the ZIP file.")

gbif-observations-dwca.zip:  11%|█         | 1.90G/17.5G [00:13<02:26, 114MB/s][codecarbon INFO @ 20:23:20] Energy consumed for RAM : 0.000858 kWh. RAM Power : 62.0 W
gbif-observations-dwca.zip:  11%|█         | 1.95G/17.5G [00:13<02:14, 123MB/s][codecarbon INFO @ 20:23:21] Delta energy consumed for CPU with cpu_load : 0.000185 kWh, power : 41.0 W
[codecarbon INFO @ 20:23:21] Energy consumed for All CPU : 0.000568 kWh
[codecarbon INFO @ 20:23:21] 0.001426 kWh of electricity used since the beginning.
gbif-observations-dwca.zip:  24%|██▍       | 4.17G/17.5G [00:28<02:56, 80.9MB/s][codecarbon INFO @ 20:23:35] Energy consumed for RAM : 0.001138 kWh. RAM Power : 62.0 W
gbif-observations-dwca.zip:  24%|██▍       | 4.22G/17.5G [00:29<02:30, 94.6MB/s][codecarbon INFO @ 20:23:36] Delta energy consumed for CPU with cpu_load : 0.000185 kWh, power : 41.0 W
[codecarbon INFO @ 20:23:36] Energy consumed for All CPU : 0.000753 kWh
[codecarbon INFO @ 20:23:36] 0.001890 kWh of electricity used since the

Extracting to 'Datos iNaturalist'...
Deleted the ZIP file.


In [22]:
df_taxonomia = pd.read_excel('2. Base de datos/Taxonomia.xlsx')

# Quitar parentesis y palabras que empiezan por mayuscula y numeros
df_taxonomia['lowered_name'] = df_taxonomia['name'].str.replace(r'[()&]', '', regex=True).apply(
    lambda x: x[0].lower() + ' '.join([w for w in x[1:].split() if w == w.lower() and not w.isdigit()])
)

In [23]:
def find_best_match(species):
    row_words = species.replace('(', '').replace(')', '').lower().split()
    if len(row_words) < 2:
        return None # No hay suficientes palabras para buscar
    # Filtrar candidatos
    candidates = df_taxonomia[df_taxonomia['lowered_name'].str.contains(' '.join(row_words[:2]))]
    # Si hay una categoría taxonómica, refinar los candidatos
    if len(row_words) > 3 and row_words[2] in ['f.', 'subsp.', 'var.']:
        candidates = candidates[candidates['lowered_name'].str.contains(f' {row_words[3]} ')]
    else:
        candidates = candidates[~candidates['lowered_name'].str.contains(r'\b(?: f\. | subsp\. | var\. )\b', regex=True)]
    if candidates.empty:
        return None
    # Calcular las puntuaciones de coincidencia
    row_words_set = set(row_words)
    candidates['score'] = candidates['lowered_name'].apply(lambda c: 
        0.9 * len(set(c.split()) & row_words_set) / len(row_words) +
        0.1 * len(set(c.split()) & row_words_set) / len(c.split())
    )
    # Obtener el mejor candidato
    best_match = candidates.loc[candidates['score'].idxmax()]
    return best_match['taxonid']

[codecarbon INFO @ 21:06:21] Energy consumed for RAM : 0.048951 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 21:06:21] Delta energy consumed for CPU with cpu_load : 0.000185 kWh, power : 41.0 W
[codecarbon INFO @ 21:06:21] Energy consumed for All CPU : 0.033871 kWh
[codecarbon INFO @ 21:06:21] 0.082823 kWh of electricity used since the beginning.


In [ ]:
# Cargar cabecera
os.makedirs('Datos iNaturalist/observations_chunked', exist_ok=True)
NUM_WORKERS_OBSERVATIONS = 64 # Porque usamos una CPU de 64 núcleos

# Guardar los encabezados en un archivo separado
pd.read_csv('Datos iNaturalist/observations.csv', chunksize=1).get_chunk(0).to_csv('Datos iNaturalist/observations_chunked/headers.csv', index=False)

def process_chunk(remainder):
    chunks = pd.read_csv('Datos iNaturalist/observations.csv', chunksize=10000)
    for i, c in enumerate(chunks):
        if (i+1) % 100 == 0:
            print(f'Processing chunk {i+1}...', end='\r')
        if i % NUM_WORKERS_OBSERVATIONS != remainder:
            continue
        df = c[(c['license'] != 'http://creativecommons.org/licenses/by-nc/4.0/') &
            (c['license'] != 'http://creativecommons.org/licenses/by-nc-sa/4.0/') &
            (c['license'] != 'http://creativecommons.org/licenses/by-nc-nd/4.0/') &
            (c['license'] != 'http://creativecommons.org/licenses/by-nc-nd/4.0/')].copy()
        df['idtaxon'] = df['scientificName'].apply(find_best_match)
        df = df[~df['idtaxon'].isna()]
        df['idtaxon'] = df['idtaxon'].astype(int)
        if len(df) > 0:
            df.to_csv(f'Datos iNaturalist/observations_chunked/{i}.csv', index=False, header=False)

# Procesar en paralelo
with ProcessPoolExecutor(max_workers=64) as executor:
    list(tqdm(executor.map(process_chunk, range(NUM_WORKERS_OBSERVATIONS)), total=NUM_WORKERS_OBSERVATIONS))

print(f'All chunks processed.')

  0%|          | 0/64 [00:00<?, ?it/s][codecarbon INFO @ 21:07:06] Energy consumed for RAM : 0.049791 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 21:07:06] Delta energy consumed for CPU with cpu_load : 0.000393 kWh, power : 87.12500000000001 W
[codecarbon INFO @ 21:07:06] Energy consumed for All CPU : 0.036115 kWh
[codecarbon INFO @ 21:07:06] 0.085905 kWh of electricity used since the beginning.
[codecarbon INFO @ 21:07:21] Energy consumed for RAM : 0.050070 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 21:07:21] Delta energy consumed for CPU with cpu_load : 0.001838 kWh, power : 407.4375 W
[codecarbon INFO @ 21:07:21] Energy consumed for All CPU : 0.037953 kWh
[codecarbon INFO @ 21:07:21] 0.088023 kWh of electricity used since the beginning.
[codecarbon INFO @ 21:07:51] Energy consumed for RAM : 0.050630 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 21:07:51] Delta energy consumed for CPU with cpu_load : 0.001839 kWh, power : 407.59125 W
[codecarbon INFO @ 21:07:51] Energy consumed for All C

Processing chunk 10200...

In [26]:
with open('Datos iNaturalist/observations_selection.csv','wb') as fout:
    # Archivo con encabezados
    fout.write(open('Datos iNaturalist/observations_chunked/headers.csv','rb').read())
    # Archivos con datos
    for filename in os.listdir('Datos iNaturalist/observations_chunked'):
        if filename == 'headers.csv':
            continue
        with open(f'Datos iNaturalist/observations_chunked/{filename}', 'rb') as fin:
            for line in fin:
                fout.write(line)
shutil.rmtree("Datos iNaturalist/observations_chunked")

[codecarbon INFO @ 22:07:53] Energy consumed for RAM : 0.117768 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 22:07:53] Delta energy consumed for CPU with cpu_load : 0.000221 kWh, power : 49.097500000000004 W
[codecarbon INFO @ 22:07:53] Energy consumed for All CPU : 0.481000 kWh
[codecarbon INFO @ 22:07:53] 0.598768 kWh of electricity used since the beginning.
[codecarbon INFO @ 22:08:08] Energy consumed for RAM : 0.118048 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 22:08:08] Delta energy consumed for CPU with cpu_load : 0.000185 kWh, power : 41.0 W
[codecarbon INFO @ 22:08:08] Energy consumed for All CPU : 0.481185 kWh
[codecarbon INFO @ 22:08:08] 0.599233 kWh of electricity used since the beginning.
[codecarbon INFO @ 22:08:23] Energy consumed for RAM : 0.118328 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 22:08:23] Delta energy consumed for CPU with cpu_load : 0.000185 kWh, power : 41.0 W
[codecarbon INFO @ 22:08:23] Energy consumed for All CPU : 0.481370 kWh
[codecarbon INFO @ 22:08:23]

In [40]:
import csv

input_path = "Datos iNaturalist/observations_selection.csv"
output_path = "Datos iNaturalist/observations_selection_cleaned.csv"

with open(input_path, newline='', encoding='utf-8') as fin, open(output_path, 'w', newline='', encoding='utf-8') as fout:
    reader = csv.reader(fin)
    writer = csv.writer(fout)

    add_idtaxon = True
    for row in reader:
        if add_idtaxon:
            row.append('idtaxon')
            add_idtaxon = False
        writer.writerow(row)

[codecarbon INFO @ 22:41:38] Energy consumed for RAM : 0.155502 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 22:41:39] Delta energy consumed for CPU with cpu_load : 0.000185 kWh, power : 41.0 W
[codecarbon INFO @ 22:41:39] Energy consumed for All CPU : 0.505959 kWh
[codecarbon INFO @ 22:41:39] 0.661461 kWh of electricity used since the beginning.
[codecarbon INFO @ 22:41:53] Energy consumed for RAM : 0.155779 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 22:41:54] Delta energy consumed for CPU with cpu_load : 0.000184 kWh, power : 41.0 W
[codecarbon INFO @ 22:41:54] Energy consumed for All CPU : 0.506144 kWh
[codecarbon INFO @ 22:41:54] 0.661923 kWh of electricity used since the beginning.
[codecarbon INFO @ 22:42:08] Energy consumed for RAM : 0.156056 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 22:42:09] Delta energy consumed for CPU with cpu_load : 0.000183 kWh, power : 41.0 W
[codecarbon INFO @ 22:42:09] Energy consumed for All CPU : 0.506327 kWh
[codecarbon INFO @ 22:42:09] 0.662383 kWh 

In [52]:
ids = pd.read_csv('Datos iNaturalist/observations_selection_cleaned.csv')['id']
len(ids)

[codecarbon INFO @ 22:45:23] Energy consumed for RAM : 0.159686 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 22:45:24] Delta energy consumed for CPU with cpu_load : 0.000184 kWh, power : 41.0 W
[codecarbon INFO @ 22:45:24] Energy consumed for All CPU : 0.508729 kWh
[codecarbon INFO @ 22:45:24] 0.668414 kWh of electricity used since the beginning.
[codecarbon INFO @ 22:45:38] Energy consumed for RAM : 0.159965 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 22:45:39] Delta energy consumed for CPU with cpu_load : 0.000185 kWh, power : 41.0 W
[codecarbon INFO @ 22:45:39] Energy consumed for All CPU : 0.508913 kWh
[codecarbon INFO @ 22:45:39] 0.668878 kWh of electricity used since the beginning.
[codecarbon INFO @ 22:45:53] Energy consumed for RAM : 0.160246 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 22:45:54] Delta energy consumed for CPU with cpu_load : 0.000186 kWh, power : 41.0 W
[codecarbon INFO @ 22:45:54] Energy consumed for All CPU : 0.509099 kWh
[codecarbon INFO @ 22:45:54] 0.669344 kWh 

2569841

In [53]:
ids

0          ## This file was auto generated by the Azure M...
1          ## Read more about the .amlignore file here: h...
2                                   .ipynb_aml_checkpoints/ 
3                                                  *.amltmp 
4          *.amltemp## This file was auto generated by th...
                                 ...                        
2569836                                             29155504
2569837                                             29155505
2569838                                             29155506
2569839                                            165539594
2569840                                            165539605
Name: id, Length: 2569841, dtype: object

[codecarbon INFO @ 22:46:08] Energy consumed for RAM : 0.160523 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 22:46:09] Delta energy consumed for CPU with cpu_load : 0.000183 kWh, power : 41.0 W
[codecarbon INFO @ 22:46:09] Energy consumed for All CPU : 0.509282 kWh
[codecarbon INFO @ 22:46:09] 0.669805 kWh of electricity used since the beginning.
[codecarbon INFO @ 22:46:23] Energy consumed for RAM : 0.160802 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 22:46:24] Delta energy consumed for CPU with cpu_load : 0.000185 kWh, power : 41.0 W
[codecarbon INFO @ 22:46:24] Energy consumed for All CPU : 0.509467 kWh
[codecarbon INFO @ 22:46:24] 0.670269 kWh of electricity used since the beginning.


In [59]:
new_ids = ids[8:] # Quitar lineas de Azure
new_ids = new_ids.apply(lambda x: str(x).replace("*.amltemp", ""))
new_ids = new_ids.astype(int)

[codecarbon INFO @ 22:51:08] Energy consumed for RAM : 0.166115 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 22:51:09] Delta energy consumed for CPU with cpu_load : 0.000185 kWh, power : 41.0 W
[codecarbon INFO @ 22:51:09] Energy consumed for All CPU : 0.512980 kWh
[codecarbon INFO @ 22:51:09] 0.679095 kWh of electricity used since the beginning.


In [ ]:
os.makedirs('Datos iNaturalist/media_chunked', exist_ok=True)
NUM_WORKERS_MEDIA = 32 # Porque 64 no caben en RAM

# Guardar los encabezados en un archivo separado
pd.read_csv('Datos iNaturalist/media.csv', chunksize=1).get_chunk(0).to_csv('Datos iNaturalist/media_chunked/headers.csv', index=False)
chunks = pd.read_csv('Datos iNaturalist/media.csv', chunksize=10000) # El argumento chunksize hace que se puedan leer archivos grandes en trozos

def process_media_chunk(remainder):
    chunks = pd.read_csv('Datos iNaturalist/media.csv', chunksize=10000)
    for i, c in enumerate(chunks):
        if (i+1) % 100 == 0:
            print(f'Processing chunk {i+1}...', end='\r')
        if i % NUM_WORKERS_MEDIA != remainder:
            continue
        df = c[c['id'].isin(new_ids) &
            (c['license'] != 'http://creativecommons.org/licenses/by-nc/4.0/') &
            (c['license'] != 'http://creativecommons.org/licenses/by-nc-sa/4.0/') &
            (c['license'] != 'http://creativecommons.org/licenses/by-nc-nd/4.0/') &
            (c['license'] != 'http://creativecommons.org/licenses/by-nc-nd/4.0/')]
        if len(df) > 0:
            df.to_csv(f'Datos iNaturalist/media_chunked/{i}.csv', index=False, header=False)

# Procesar en paralelo
with ProcessPoolExecutor(max_workers=NUM_WORKERS_MEDIA) as executor:
    list(tqdm(executor.map(process_media_chunk, range(NUM_WORKERS_MEDIA)), total=NUM_WORKERS_MEDIA))

print(f'All chunks processed.')

In [62]:
with open('Datos iNaturalist/media_selection.csv','wb') as fout:
    # Archivo con encabezados
    fout.write(open('Datos iNaturalist/media_chunked/headers.csv','rb').read())
    # Archivos con datos
    for filename in os.listdir('Datos iNaturalist/media_chunked'):
        if filename == 'headers.csv':
            continue
        with open(f'Datos iNaturalist/media_chunked/{filename}', 'rb') as fin:
            for line in fin:
                fout.write(line)
shutil.rmtree("Datos iNaturalist/media_chunked")

[codecarbon INFO @ 23:12:38] Energy consumed for RAM : 0.190160 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 23:12:39] Delta energy consumed for CPU with cpu_load : 0.000185 kWh, power : 41.0 W
[codecarbon INFO @ 23:12:39] Energy consumed for All CPU : 0.553266 kWh
[codecarbon INFO @ 23:12:39] 0.743426 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:12:39] 0.004057 g.CO2eq/s mean an estimation of 127.93206759107171 kg.CO2eq/year
[codecarbon INFO @ 23:12:53] Energy consumed for RAM : 0.190440 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 23:12:54] Delta energy consumed for CPU with cpu_load : 0.000185 kWh, power : 41.0 W
[codecarbon INFO @ 23:12:54] Energy consumed for All CPU : 0.553451 kWh
[codecarbon INFO @ 23:12:54] 0.743890 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:13:08] Energy consumed for RAM : 0.190719 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 23:13:09] Delta energy consumed for CPU with cpu_load : 0.000185 kWh, power : 41.0 W
[codecarbon

OSError: [Errno 39] Directory not empty: 'Datos iNaturalist/media_chunked'

In [65]:
observations_df = pd.read_csv('Datos iNaturalist/observations_selection_cleaned.csv')
media_df = pd.read_csv('Datos iNaturalist/media_selection.csv')
observations_df = observations_df[observations_df['id'].isin(media_df['id'].unique())]
observations_df.to_csv('Datos iNaturalist/observations_selection_final.csv', index=False)
len(observations_df)

[codecarbon INFO @ 23:21:23] Energy consumed for RAM : 0.199946 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 23:21:24] Delta energy consumed for CPU with cpu_load : 0.000185 kWh, power : 41.0 W
[codecarbon INFO @ 23:21:24] Energy consumed for All CPU : 0.559738 kWh
[codecarbon INFO @ 23:21:24] 0.759684 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:21:38] Energy consumed for RAM : 0.200226 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 23:21:39] Delta energy consumed for CPU with cpu_load : 0.000185 kWh, power : 41.0 W
[codecarbon INFO @ 23:21:39] Energy consumed for All CPU : 0.559923 kWh
[codecarbon INFO @ 23:21:39] 0.760148 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:21:53] Energy consumed for RAM : 0.200505 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 23:21:54] Delta energy consumed for CPU with cpu_load : 0.000185 kWh, power : 41.0 W
[codecarbon INFO @ 23:21:54] Energy consumed for All CPU : 0.560108 kWh
[codecarbon INFO @ 23:21:54] 0.760613 kWh 

413038

[codecarbon INFO @ 23:22:23] Energy consumed for RAM : 0.201063 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 23:22:24] Delta energy consumed for CPU with cpu_load : 0.000184 kWh, power : 41.0 W
[codecarbon INFO @ 23:22:24] Energy consumed for All CPU : 0.560477 kWh
[codecarbon INFO @ 23:22:24] 0.761540 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:22:38] Energy consumed for RAM : 0.201343 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 23:22:39] Delta energy consumed for CPU with cpu_load : 0.000185 kWh, power : 41.0 W
[codecarbon INFO @ 23:22:39] Energy consumed for All CPU : 0.560662 kWh
[codecarbon INFO @ 23:22:39] 0.762004 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:22:39] 0.004054 g.CO2eq/s mean an estimation of 127.83905633998748 kg.CO2eq/year
[codecarbon INFO @ 23:22:53] Energy consumed for RAM : 0.201622 kWh. RAM Power : 62.0 W
[codecarbon INFO @ 23:22:54] Delta energy consumed for CPU with cpu_load : 0.000185 kWh, power : 41.0 W
[codecarbon

### Vemos cuántas especies nos quedan después de descargar las observaciones de todo el mundo

In [ ]:
import pandas as pd

In [ ]:
taxonomia_df = pd.read_excel('../../Taxonomia.xlsx')
inaturalist_df = pd.read_csv('../../../1. Obtención Y Procesado de Datos/Ubicaciones y media/Datos iNaturalist/observations_processed.csv')
inat = inaturalist_df['idtaxon'].dropna().astype(int).drop_duplicates()
df = pd.read_csv("./Datos iNaturalist/observations_selection_final.csv")
all_inaturalist = pd.concat([inat, df['idtaxon'].dropna().astype(int).drop_duplicates()]).drop_duplicates().sort_values()

C:\Users\alber\AppData\Local\Temp\ipykernel_5076\771626618.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Datos iNaturalist/observations_selection_final.csv")


In [ ]:
len(set(taxonomia_df[~taxonomia_df['taxonid'].isin(all_inaturalist)]['taxonid']))

946

In [ ]:
sin_foto = taxonomia_df[~taxonomia_df['taxonid'].isin(all_inaturalist)]
sin_foto = sin_foto[sin_foto['nametype'] == 'Aceptado']
sin_foto.to_excel('../Especies_Sin_Foto_iNaturalist.xlsx', index=False)

In [ ]:
# Fin del proceso de medición
emissions = tracker.stop()

# Accede a las emisiones calculadas
print(f"Emissions: {emissions} kg CO₂eq")

[codecarbon INFO @ 23:22:56] Energy consumed for RAM : 0.201656 kWh. RAM Power : 62.0 W


Emissions: 0.0998906981700295 kg CO₂eq
